# 🚞 Zero-shot RE Training

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# # if you're running this in a colab notebook, you can run this cell to install the necessary dependencies
# !git clone https://github.com/jackboyla/GLiREL.git
# !cd GLiREL && pip install -e .
# !python -m spacy download en_core_web_sm

In [3]:
from glirel import GLiREL

save_path = 'logs/zero_rel/zero_rel-2024-05-29__13-31-21/model_1000'
model = GLiREL.from_pretrained(save_path)
# model

2024-05-29 14:24:35,493 - datasets - INFO - PyTorch version 2.3.0 available.
config.json not found in /home/jackboylan/GLiREL/logs/zero_rel/zero_rel-2024-05-29__13-31-21/model_1000
2024-05-29 14:24:35,914 - huggingface_hub.hub_mixin - WARNING - config.json not found in /home/jackboylan/GLiREL/logs/zero_rel/zero_rel-2024-05-29__13-31-21/model_1000
/opt/conda/envs/glirel/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/opt/conda/envs/glirel/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and

# Inference

To infer, the model needs `tokens`, `NER`, and `zero shot labels`.

### Eval data

In [4]:
import json
with open('./data/few_rel_eval.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

i = 0

tokens = data[i]['tokenized_text']
ner = data[i]['ner']
labels = list(set([r['relation_text'] for r in data[i]['relations']]))
print(tokens)
print()
print(ner)
print(labels)

['The', 'race', 'took', 'place', 'between', 'Godstow', 'and', 'Binsey', 'along', 'the', 'Upper', 'River', 'Thames', '.']

[[7, 8, 'Q4914513', 'Binsey'], [11, 13, 'Q19686', 'River Thames']]
['located in or next to body of water']


In [5]:
labels = ['country of origin', 'licensed to broadcast to', 'father', 'followed by'] + labels
print(labels)

['country of origin', 'licensed to broadcast to', 'father', 'followed by', 'located in or next to body of water']


In [6]:
relations = model.predict_relations(tokens, labels, threshold=0.01, ner=ner)

print('Number of relations:', len(relations))  # num entity pairs (both directions) * num classes.... provided they're over the threshold

sorted_data_desc = sorted(relations, key=lambda x: x['score'], reverse=True)
print("\nDescending Order by Score:")
for item in sorted_data_desc:
    print(item)

Number of relations: 10

Descending Order by Score:
{'head_pos': [7, 8], 'tail_pos': [11, 13], 'head_text': ['Binsey'], 'tail_text': ['River', 'Thames'], 'label': 'followed by', 'score': 0.032704684883356094}
{'head_pos': [11, 13], 'tail_pos': [7, 8], 'head_text': ['River', 'Thames'], 'tail_text': ['Binsey'], 'label': 'followed by', 'score': 0.032640282064676285}
{'head_pos': [7, 8], 'tail_pos': [11, 13], 'head_text': ['Binsey'], 'tail_text': ['River', 'Thames'], 'label': 'father', 'score': 0.03219030424952507}
{'head_pos': [11, 13], 'tail_pos': [7, 8], 'head_text': ['River', 'Thames'], 'tail_text': ['Binsey'], 'label': 'father', 'score': 0.03211749717593193}
{'head_pos': [7, 8], 'tail_pos': [11, 13], 'head_text': ['Binsey'], 'tail_text': ['River', 'Thames'], 'label': 'country of origin', 'score': 0.02798185497522354}
{'head_pos': [11, 13], 'tail_pos': [7, 8], 'head_text': ['River', 'Thames'], 'tail_text': ['Binsey'], 'label': 'country of origin', 'score': 0.02792412042617798}
{'head_p

### Real-world example

In [9]:
# Real-world example
import spacy
nlp = spacy.load('en_core_web_sm')

text = "Apple Inc. was founded by Steve Jobs, Steve Wozniak, and Ronald Wayne in April 1976. The company is headquartered in Cupertino, California."

# text = "Jack Dorsey's father, Tim Dorsey, is a licensed pilot. Jack met his wife Sarah Paulson in New York in 2003. They have one son, Edward."

doc = nlp(text)

labels = ['country of origin', 'licensed to broadcast to', 'parent', 'followed by', 'located in or next to body of water', 'spouse', 'child', 'founder', 'headquartered in', 'acquired by', 'subsidiary of']

tokens = [token.text for token in doc]

ner = [[ent.start, ent.end, ent.label_, ent.text] for ent in doc.ents]
print(f"Entities detected: {ner}")

relations = model.predict_relations(tokens, labels, threshold=0.01, ner=ner)

print('Number of relations:', len(relations))

sorted_data_desc = sorted(relations, key=lambda x: x['score'], reverse=True)
print("\nDescending Order by Score:")
for item in sorted_data_desc:
    print(f"{item['head_text']} --> {item['label']} --> {item['tail_text']} | score: {item['score']}")

Entities detected: [[0, 2, 'ORG', 'Apple Inc.'], [5, 7, 'PERSON', 'Steve Jobs'], [8, 10, 'PERSON', 'Steve Wozniak'], [12, 14, 'PERSON', 'Ronald Wayne'], [15, 17, 'DATE', 'April 1976'], [23, 24, 'GPE', 'Cupertino'], [25, 26, 'GPE', 'California']]
Number of relations: 462

Descending Order by Score:
['Apple', 'Inc.'] --> founder --> ['California'] | score: 0.04158327355980873
['Apple', 'Inc.'] --> founder --> ['Cupertino'] | score: 0.04156165197491646
['Apple', 'Inc.'] --> founder --> ['Steve', 'Jobs'] | score: 0.04155745357275009
['Apple', 'Inc.'] --> founder --> ['Ronald', 'Wayne'] | score: 0.04155644774436951
['California'] --> founder --> ['Apple', 'Inc.'] | score: 0.04154440760612488
['California'] --> founder --> ['Cupertino'] | score: 0.0415400005877018
['Apple', 'Inc.'] --> founder --> ['Steve', 'Wozniak'] | score: 0.041534148156642914
['Cupertino'] --> founder --> ['Apple', 'Inc.'] | score: 0.04152711480855942
['Cupertino'] --> founder --> ['California'] | score: 0.0415237545967

In [8]:
import spacy
from spacy.tokens import Span
from spacy import displacy

def visualize_relation(text, relations):
    nlp = spacy.blank("en")
    doc = nlp(text)

    # Manually set dependency relations to visualize relations
    for token in doc:
        token.dep_ = "dep"  # default to 'dep'

    spans = []
    # Calculate character offsets for each entity
    for rel in relations:
        head = Span(doc, rel['head_pos'][0], rel['head_pos'][1], label=rel['head_text'])
        tail = Span(doc, rel['tail_pos'][0], rel['tail_pos'][1], label=rel['tail_text'])

        doc.ents += (head, tail)

        # Mock dependencies
        head_root = head.root
        tail_root = tail.root

        head_root.dep_ = "rel"  # Relation type can be customized
        head_root.head = tail_root  # Point head to tail

    options = {"fine_grained": True}
    displacy.render(doc, style="dep", options=options, jupyter=True)

# Example data
text = "Binsey located in or next to body of water River Thames"
relations = [
    {'head_pos': [0, 1], 'tail_pos': [9, 11], 'head_text': 'Binsey', 'tail_text': 'River Thames', 'label': 'located in or next to body of water', 'score': 0.9235768914222717},
    # {'head_pos': [9, 11], 'tail_pos': [0, 1], 'head_text': 'River Thames', 'tail_text': 'Binsey', 'label': 'located in or next to body of water', 'score': 0.12615662813186646}
]

visualize_relation(text, relations)
